In [2]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus
import requests

In [3]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [6]:
df_comp = pd.DataFrame(list(db.EUROSTAT_GOV10A_EXP_GDP_COMPARATIVA.find()))

In [7]:
df_comp.head()

,_id,year,education_pct_gdp,pensions_pct_gdp,country
0,685cfc7c9b9ce874eaa20f6e,1995,4.3,6.6,ES
1,685cfc7c9b9ce874eaa20f6f,1996,4.3,6.7,ES
2,685cfc7c9b9ce874eaa20f70,1997,4.2,6.6,ES
3,685cfc7c9b9ce874eaa20f71,1998,4.1,6.5,ES
4,685cfc7c9b9ce874eaa20f72,1999,4.2,6.4,ES


In [8]:
df_comp = df_comp.sort_values(['country','year'])
df_comp['delta_edu'] = df_comp.groupby('country')['education_pct_gdp'].diff()

In [9]:
df_comp.head()

,_id,year,education_pct_gdp,pensions_pct_gdp,country,delta_edu
29,685cfc7c9b9ce874eaa20f8b,1995,5.5,12.1,AT,NaN
30,685cfc7c9b9ce874eaa20f8c,1996,5.5,12.3,AT,0.0
31,685cfc7c9b9ce874eaa20f8d,1997,5.5,12.2,AT,0.0
32,685cfc7c9b9ce874eaa20f8e,1998,5.4,12.1,AT,-0.1
33,685cfc7c9b9ce874eaa20f8f,1999,5.4,12.2,AT,0.0


In [10]:
df_sf = pd.DataFrame(list(db.EUROSTAT_SPENDING_FLOWS.find()))
df_sf.head()

,_id,geo,time_period,gf01_general_public_services,gf07_health,gf09_education,gf1002_old_age,d51c1,d51c2,d51c3,d51d,d51e,d59,d61,d91,inclusion_index,urban_rate
0,685d23d139e248e523f55877,DE,1995,7.4,6.1,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,685d23d139e248e523f55878,DE,1996,7.3,6.2,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,685d23d139e248e523f55879,DE,1997,7.3,6.1,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,685d23d139e248e523f5587a,DE,1998,7.2,6.1,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,685d23d139e248e523f5587b,DE,1999,6.9,6.1,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [12]:
df_sf_simple = df_sf[['geo','time_period','inclusion_index','urban_rate']]

In [16]:
df_sf_simple = (
    df_sf
      .rename(columns={
         'geo': 'country',
         'time_period': 'year'    # cambia time_period a year
      })[
         ['country','year','inclusion_index','urban_rate']
      ]
)

# Convertir year de string a int
df_sf_simple['year'] = df_sf_simple['year'].astype(int)

In [17]:
df_sf_simple.sample(5)

,country,year,inclusion_index,urban_rate
59,EU27_2020,1996,NaN,NaN
94,IT,2002,NaN,NaN
32,ES,1998,NaN,NaN
123,SE,2002,NaN,NaN
142,SE,2021,99.375,88.238


In [18]:
df_mod = pd.merge(
    df_comp,
    df_sf_simple,
    on=['country','year'],
    how='inner'
)

In [19]:
df_mod

,_id,year,education_pct_gdp,pensions_pct_gdp,country,delta_edu,inclusion_index,urban_rate
0,685cfc7c9b9ce874eaa20f6e,1995,4.3,6.6,ES,NaN,NaN,NaN
1,685cfc7c9b9ce874eaa20f6f,1996,4.3,6.7,ES,0.0,NaN,NaN
2,685cfc7c9b9ce874eaa20f70,1997,4.2,6.6,ES,-0.1,NaN,NaN
3,685cfc7c9b9ce874eaa20f71,1998,4.1,6.5,ES,-0.1,NaN,NaN
4,685cfc7c9b9ce874eaa20f72,1999,4.2,6.4,ES,0.1,NaN,NaN
5,685cfc7c9b9ce874eaa20f73,2000,4.1,6.3,ES,-0.1,NaN,NaN
6,685cfc7c9b9ce874eaa20f74,2001,4.0,6.1,ES,-0.1,NaN,NaN
7,685cfc7c9b9ce874eaa20f75,2002,4.0,6.2,ES,0.0,NaN,NaN
8,685cfc7c9b9ce874eaa20f76,2003,4.0,6.1,ES,0.0,NaN,NaN
9,685cfc7c9b9ce874eaa20f77,2004,4.1,6.0,ES,0.1,NaN,NaN


In [21]:
df_mod = df_mod.drop('_id', axis = 1)

In [22]:
df_mod.sample()

,year,education_pct_gdp,pensions_pct_gdp,country,delta_edu,inclusion_index,urban_rate
3,1998,4.1,6.5,ES,-0.1,NaN,NaN


In [23]:
coll = db['MODERNIZATION_METRICS']
coll.drop()
coll.insert_many(df_mod.to_dict('records'))
print("Registros en la colección: ", coll.count_documents({}))
print("Número de filas de df_raw: ", len(df_mod))

Registros en la colección:  58
Número de filas de df_raw:  58
